In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import gc

In [2]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

# Memory Reduction

In [3]:
# def reduce_mem_usage(props):
#     start_mem_usg = props.memory_usage().sum() / 1024**2 
#     print("Memory usage of properties dataframe is :",start_mem_usg," MB")
#     NAlist = [] # Keeps track of columns that have missing values filled in. 
#     for col in props.columns:
#         if props[col].dtype != object:  # Exclude strings
            
#             # Print current column type
#             #print("******************************")
#             #print("Column: ",col)
#             #print("dtype before: ",props[col].dtype)
            
#             # make variables for Int, max and min
#             IsInt = False
#             mx = props[col].max()
#             mn = props[col].min()
            
#             # Integer does not support NA, therefore, NA needs to be filled
#             if not np.isfinite(props[col]).all(): 
#                 NAlist.append(col)
#                 props[col].fillna(mn-1,inplace=True)  
                   
#             # test if column can be converted to an integer
#             asint = props[col].fillna(0).astype(np.int64)
#             result = (props[col] - asint)
#             result = result.sum()
#             if result > 0.01:
#                 IsInt = True

            
#             # Make Integer/unsigned Integer datatypes
#             if IsInt:
#                 if mn >= 0:
#                     if mx < 255:
#                         props[col] = props[col].astype(np.uint8)
#                     elif mx < 65535:
#                         props[col] = props[col].astype(np.uint16)
#                     elif mx < 4294967295:
#                         props[col] = props[col].astype(np.uint32)
#                     else:
#                         props[col] = props[col].astype(np.uint64)
#                 else:
#                     if mn > np.iinfo(np.int8).max:
#                         props[col] = props[col].astype(np.int8)
#                     elif mn > np.iinfo(np.int16).max:
#                         props[col] = props[col].astype(np.int16)
#                     elif mn > np.iinfo(np.int32).max:
#                         props[col] = props[col].astype(np.int32)
#                     elif mn > np.iinfo(np.int64).max:
#                         props[col] = props[col].astype(np.int64)    
            
#             # Make float datatypes 32 bit
#             else:
#                 props[col] = props[col].astype(np.float32)
            
#             # Print new column type
#             #print("dtype after: ",props[col].dtype)
#             #print("******************************")
    
#     # Print final result
#     print("___MEMORY USAGE AFTER COMPLETION:___")
#     mem_usg = props.memory_usage().sum() / 1024**2 
#     print("Memory usage is: ",mem_usg," MB")
#     print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
#     return props, NAlist

In [4]:
# train, NAlist = reduce_mem_usage(train)
# print("_________________")
# print("Warning: the following columns have missing values filled with 'df['column_name'].min() -1': ")
# print("_________________")
# print("")
# print(NAlist)

# Train Data

In [5]:
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67091776 entries, 0 to 67091775
Data columns (total 7 columns):
game_id           int64
winner            int64
time              float64
player            int64
species           object
event             object
event_contents    object
dtypes: float64(1), int64(3), object(3)
memory usage: 3.5+ GB
None


In [6]:
train.head()

,game_id,winner,time,player,species,event,event_contents
0,0,1,0.00,0,T,Camera,"at (145.25, 21.5078125)"
1,0,1,0.00,1,T,Camera,"at (22.75, 147.0078125)"
2,0,1,0.02,0,T,Selection,['OrbitalCommand [3080001]']
3,0,1,0.02,0,T,Ability,(1360) - TrainSCV
4,0,1,0.14,0,T,Camera,"at (142.99609375, 24.50390625)"


In [7]:
train['game_id']=train['game_id'].astype(np.uint8)
train['winner']=train['winner'].astype(np.uint8)
train['time']=train['time'].astype(np.float32)
train['player']=train['player'].astype(np.uint8)

In [8]:
train[['species']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67091776 entries, 0 to 67091775
Data columns (total 1 columns):
species    object
dtypes: object(1)
memory usage: 511.9+ MB


In [9]:
species_dict = {i:j for (i,j) in zip(train.species.unique(), range(train.species.nunique()))}
print(species_dict)
train['species'] = train['species'].map(species_dict)

{'T': 0, 'P': 1, 'Z': 2}


In [10]:
train['species']=train['species'].astype(np.uint8)

In [11]:
train[['species']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67091776 entries, 0 to 67091775
Data columns (total 1 columns):
species    uint8
dtypes: uint8(1)
memory usage: 64.0 MB


In [12]:
event_dict = {i:j for (i,j) in zip(train.event.unique(), range(train.event.nunique()))}
print(event_dict)
train['event'] = train['event'].map(event_dict)

{'Camera': 0, 'Selection': 1, 'Ability': 2, 'Right Click': 3, 'SetControlGroup': 4, 'GetControlGroup': 5, 'AddToControlGroup': 6, 'ControlGroup': 7}


In [13]:
train[['event']]=train[['event']].astype(np.uint8)

In [14]:
train[['event']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67091776 entries, 0 to 67091775
Data columns (total 1 columns):
event    uint8
dtypes: uint8(1)
memory usage: 64.0 MB


- 0: Camera
- 1: Selection
- 2: Ability
- 3: RightClick
- 4: SetControlGroup
- 5: GetControlGroup
- 6: AddToControGroup
- 7: ControlGroup

In [15]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67091776 entries, 0 to 67091775
Data columns (total 7 columns):
game_id           uint8
winner            uint8
time              float32
player            uint8
species           uint8
event             uint8
event_contents    object
dtypes: float32(1), object(1), uint8(5)
memory usage: 1.1+ GB


In [16]:
%%time
train.to_pickle('train_modified.pkl')
#train.to_csv('train_modified.csv', index=False)

CPU times: user 16.4 s, sys: 4.48 s, total: 20.9 s
Wall time: 21 s


In [17]:
import pickle
with open('train_modified.pkl', 'rb') as f:
    data_train = pickle.load(f)

In [18]:
data_train.head()

,game_id,winner,time,player,species,event,event_contents
0,0,1,0.00,0,0,0,"at (145.25, 21.5078125)"
1,0,1,0.00,1,0,0,"at (22.75, 147.0078125)"
2,0,1,0.02,0,0,1,['OrbitalCommand [3080001]']
3,0,1,0.02,0,0,2,(1360) - TrainSCV
4,0,1,0.14,0,0,0,"at (142.99609375, 24.50390625)"


In [ ]:
data_trainin.info()

# Test Data

In [19]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28714849 entries, 0 to 28714848
Data columns (total 6 columns):
game_id           int64
time              float64
player            int64
species           object
event             object
event_contents    object
dtypes: float64(1), int64(2), object(3)
memory usage: 1.3+ GB


In [20]:
test['game_id']=train['game_id'].astype(np.uint8)
test['player']=train['player'].astype(np.uint8)
test['time']=train['time'].astype(np.float32)

In [21]:
test[['species']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28714849 entries, 0 to 28714848
Data columns (total 1 columns):
species    object
dtypes: object(1)
memory usage: 219.1+ MB


In [22]:
species_dict = {i:j for (i,j) in zip(test.species.unique(), range(test.species.nunique()))}
print(species_dict)
test['species'] = test['species'].map(species_dict)

{'P': 0, 'Z': 1, 'T': 2}


In [23]:
test['species']=test['species'].astype(np.uint8)

In [24]:
test[['species']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28714849 entries, 0 to 28714848
Data columns (total 1 columns):
species    uint8
dtypes: uint8(1)
memory usage: 27.4 MB


In [25]:
event_dict = {i:j for (i,j) in zip(test.event.unique(), range(test.event.nunique()))}
print(event_dict)
test['event'] = test['event'].map(event_dict)

{'Camera': 0, 'Selection': 1, 'Ability': 2, 'AddToControlGroup': 3, 'SetControlGroup': 4, 'Right Click': 5, 'GetControlGroup': 6, 'ControlGroup': 7}


{'Camera': 0, 'Selection': 1, 'Ability': 2, 'Right Click': 3, 'SetControlGroup': 4, 'GetControlGroup': 5, 'AddToControlGroup': 6, 'ControlGroup': 7}

In [26]:
test[['event']]=test[['event']].astype(np.uint8)

In [27]:
test[['event']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28714849 entries, 0 to 28714848
Data columns (total 1 columns):
event    uint8
dtypes: uint8(1)
memory usage: 27.4 MB


In [28]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28714849 entries, 0 to 28714848
Data columns (total 6 columns):
game_id           uint8
time              float32
player            uint8
species           uint8
event             uint8
event_contents    object
dtypes: float32(1), object(1), uint8(4)
memory usage: 438.2+ MB


In [29]:
%%time
test.to_pickle('test_modified.pkl')
#test.to_csv('test_modified.csv', index=False)

CPU times: user 6.66 s, sys: 1.39 s, total: 8.05 s
Wall time: 8.03 s


In [30]:
import pickle
with open('test_modified.pkl', 'rb') as f:
    data_test = pickle.load(f)

In [31]:
data_test.head()

,game_id,time,player,species,event,event_contents
0,0,0.00,0,0,0,"at (22.25, 81.5078125)"
1,0,0.00,1,0,0,"at (120.25, 153.83984375)"
2,0,0.02,0,0,1,['Nexus [3100001]']
3,0,0.02,0,0,2,(15E0) - TrainProbe
4,0,0.14,0,0,3,NaN


In [32]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28714849 entries, 0 to 28714848
Data columns (total 6 columns):
game_id           uint8
time              float32
player            uint8
species           uint8
event             uint8
event_contents    object
dtypes: float32(1), object(1), uint8(4)
memory usage: 438.2+ MB
